<a href="https://colab.research.google.com/github/RobsonMartins27/aurora-varejo-analytics/blob/main/analise_vendas_aurora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Configuração e Importação de Bibliotecas

Preparação do ambiente de trabalho com as ferramentas necessárias

In [ ]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

## 2. Geração e Carga da Base de Dados Bruta
Simulação do ecossistema de vendas da Aurora Varejo para fins de estudo e auditoria.


In [ ]:
# @title
# Configuração de reprodutibilidade
np.random.seed(42)

# Gerando dados
n_linhas = 1000
data_inicial = datetime(2025, 1, 1)

datas = [data_inicial + timedelta(days=random.randint(0, 365)) for _ in range(n_linhas)]
categorias = ['Eletrônicos', 'Moda', 'Casa', 'Beleza', 'Esportes']
vendas = np.random.normal(250, 100, n_linhas).round(2)
# Inserindo erros propositais (valores negativos)
vendas[0:10] = -50

custos = vendas * 0.6 + np.random.normal(10, 5, n_linhas)
regioes = ['Norte', 'Sul', 'Leste', 'Oeste', 'Centro-Oeste']

df_simulado = pd.DataFrame({
    'id_pedido': range(1001, 1001 + n_linhas),
    'data': datas,
    'cliente_id': [random.randint(5000, 5500) for _ in range(n_linhas)],
    'categoria': [random.choice(categorias) for _ in range(n_linhas)],
    'valor_venda': vendas,
    'custo_produto': custos,
    'regiao': [random.choice(regioes) for _ in range(n_linhas)]
})

# Salvando o arquivo
df_simulado.to_csv('vendas_aurora.csv', index=False)
print("Arquivo 'vendas_aurora.csv' gerado com sucesso!")

Arquivo 'vendas_aurora.csv' gerado com sucesso!


## 3. Exploração e Diagnóstico dos Dados

Análise preliminar da estrutura da base, identificação de tipos de dados e detecção de inconsistências (valores nulos ou negativos).


In [ ]:
# @title
# Carreguei  os dados atribuíndo a uma varíavel.
df_bruto = pd.read_csv('vendas_aurora.csv')

# EXPLORANDO OS DADOS
print("--- VISÃO GERAL(PRIMEIRAS LINHAS) ---")
display(df_bruto.head())

print("--- INFORMAÇÕES ---")
df_bruto.info()

print("--- RESUMO ESTATÍSTICO ---")
display(df_bruto.describe())

print("--- FREQUÊNCIA POR CATEGORIA ---")
display(df_bruto['categoria'].value_counts())

print("--- RANKING DE VENDAS POR REGIÃO ---")
display(df_bruto['regiao'].value_counts())

--- VISÃO GERAL(PRIMEIRAS LINHAS) ---


,id_pedido,data,cliente_id,categoria,valor_venda,custo_produto,regiao
0,1001,2025-03-16,5445,Moda,-50.0,-13.003223,Centro-Oeste
1,1002,2025-03-03,5192,Moda,-50.0,-15.376832,Norte
2,1003,2025-06-04,5141,Moda,-50.0,-19.701848,Centro-Oeste
3,1004,2025-07-21,5126,Moda,-50.0,-23.234684,Oeste
4,1005,2025-05-19,5500,Moda,-50.0,-16.508883,Centro-Oeste


--- INFORMAÇÕES ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id_pedido      1000 non-null   int64  
 1   data           1000 non-null   object 
 2   cliente_id     1000 non-null   int64  
 3   categoria      1000 non-null   object 
 4   valor_venda    1000 non-null   float64
 5   custo_produto  1000 non-null   float64
 6   regiao         1000 non-null   object 
dtypes: float64(2), int64(2), object(3)
memory usage: 54.8+ KB
--- RESUMO ESTATÍSTICO ---


,id_pedido,cliente_id,valor_venda,custo_produto
count,1000.000000,1000.00000,1000.00000,1000.000000
mean,1500.500000,5254.36900,248.48532,159.445373
std,288.819436,144.61617,102.09711,61.205954
min,1001.000000,5000.00000,-74.13000,-31.784220
25%,1250.750000,5129.00000,182.21000,119.570537
50%,1500.500000,5250.50000,251.28000,161.984776
75%,1750.250000,5381.00000,314.12750,199.278338
max,2000.000000,5500.00000,635.27000,399.909884


--- FREQUÊNCIA POR CATEGORIA ---


,count
categoria,
Esportes,230
Eletrônicos,206
Moda,195
Casa,191
Beleza,178


--- FREQUÊNCIA POR REGIÃO ---


,count
regiao,
Oeste,209
Sul,207
Leste,200
Centro-Oeste,193
Norte,191


## 4. Auditoria de Qualidade e Relatório de Erros
Isolamento e análise de registros inconsistentes para entender a origem das falhas no registro de vendas por categoria.


In [ ]:
# Criando um DataFrame exclusivo para análise dos erros (vendas negativas)
# Usamos .copy() para garantir que este seja um objeto independente do dado bruto
df_erros = df_bruto[df_bruto['valor_venda'] < 0].copy()

# Calculando a quantidade total de registros inconsistentes encontrados
total_erros = len(df_erros)
print(total_erros)

# Ranking de erros por categoria para identificar a origem do problema
display(df_erros['categoria'].value_counts())

14


,count
categoria,
Moda,7
Casa,3
Beleza,2
Esportes,1
Eletrônicos,1


💡 Insight: Notei que a categoria 'Moda' concentra a maior parte das inconsistências (7 registros). Isso indica que o problema não é aleatório, mas pode estar no processo de checkout ou no sistema de entrada de dados específico desse setor.

## 5. Limpeza, Tipagem e Feature Engineering
Filtragem definitiva de erros, conversão de tipos (datetime) e criação de métricas de negócio como 'Lucro' e segmentação por período (Mês/Ano).


In [ ]:

df_limpo = df_bruto[df_bruto['valor_venda'] > 0].copy()

print("Estatísticas da coluna valor venda (pós limpeza):")
display(df_limpo['valor_venda'].describe())

# - Converti a data que está como objeto/string para o tipo de data
df_limpo['data'] = pd.to_datetime(df_limpo['data'])

#df_limpo.info() # - Conferindo se deu certo a conversão

df_limpo['lucro'] = df_limpo['valor_venda'] - df_limpo['custo_produto'] # - Criando um campo de lucro.

#display(df_limpo.head()) # - Conferindo a criação do campo

media_lucro = df_limpo['lucro'].mean()
print(f"O lucro médio por venda é de R$ {media_lucro:.2f}")

df_limpo['ano'] = df_limpo['data'].dt.year
df_limpo['mes'] = df_limpo['data'].dt.month

display(df_limpo[['valor_venda', 'custo_produto', 'lucro','ano', 'mes']].head())

Estatísticas da coluna valor venda (pós limpeza):


,valor_venda
count,986.000000
mean,252.643215
std,96.602710
min,2.840000
25%,185.377500
50%,252.650000
75%,314.790000
max,635.270000


O lucro médio por venda é de R$ 90.73


,valor_venda,custo_produto,lucro,ano,mes
10,203.66,138.782970,64.877030,2025,8
11,203.43,133.045998,70.384002,2025,8
12,274.20,184.896304,89.303696,2025,1
13,58.67,41.756061,16.913939,2025,12
14,77.51,65.185819,12.324181,2025,10


## 6. Exportação da Base Tratada para Visualização
Geração do arquivo final saneado para consumo em ferramentas de Business Intelligence (BI).



In [ ]:
df_limpo.to_csv('vendas_aurora_final.csv', index=False)

print("Arquivo gerado com sucesso!")

Arquivo gerado com sucesso!
